#### __HOMEWORK 2__

##### __Problem 1__

Let $S = \$100$, $K = \$105$, $r = 8\%$, $T = 0.5$, and $\delta = 0.0$. Let $u = 1.3$, $d = 0.8$, and $n = 1$.

What are the premium, $\Delta$, and $B$ for a European call?  
What are the premium, $\Delta$, and $B$ for a European put?


In [ ]:
def euroBinomPricerRecursive(S, K, r, v, q, T, n, payoff, verbose = True):
    
    nodes = n + 1
    h = T / n
    u = np.exp((r - q) * h + v * np.sqrt(h))
    d = np.exp((r - q) * h - v * np.sqrt(h))
    pu = (np.exp((r - q) * h) - d) / (u - d)
    pd = 1.0 - pu
    disc = np.exp(-r * h)
    
    
    ## Arrays to store the spot prices and option values
    Ct = np.empty(nodes)
    St = np.empty(nodes)
    
    for i in range(nodes):
        St[i] = S * (u ** (n - i)) * (d ** i)
        Ct[i] = payoff(St[i], K)
    
    if verbose:
        print(Ct)
        
    for t in range((n - 1), -1, -1):
        for j in range(t+1):
            Ct[j] = disc * (pu * Ct[j] + pd * Ct[j+1])
            # St[j] = St[j] / u
            # Ct[j] = np.maximum(Ct[j], early payoff)
            print(Ct)
            
    return Ct[0]

In [14]:
import numpy as np
from scipy.stats import binom

def callPayoff(spot, strike):
    return np.maximum(spot - strike, 0.0)

## Put Payoff Function
def putPayoff(spot, strike):
    return np.maximum(strike - spot, 0.0)

#### __The Simplified European Binomial Pricer__

In [2]:
def euroBinomPricer(S, K, r, v, q, T, n, payoff, verbose = True):
    nodes = n  + 1
    h = T / n
    u = np.exp((r - q) * h + v * np.sqrt(h))
    d = np.exp((r - q) * h - v * np.sqrt(h))
    pstar = (np.exp((r - q) * h) - d) / (u - d)
    
    price = 0.0
    
    for i in range(nodes):
        prob = binom.pmf(i, n, pstar)
        spotT = S * (u ** i) * (d ** (n - i))
        po = payoff(spotT, K) 
        price += po * prob
        if verbose:
            print(f"({spotT:0.4f}, {po:0.4f}, {prob:0.4f})")
        
    price *= np.exp(-r * T)
    
    return price

In [3]:
S = 41.0
K = 40.0
r = 0.08
v = 0.30
q = 0.0
T = 1.0
n = 3

putPrc = euroBinomPricer(S, K, r, v, q, T, n, putPayoff, verbose = True)
print(f"The European Put Premium: {putPrc : 0.3f}")

(26.4157, 13.5843, 0.1603)
(37.3513, 2.6487, 0.4044)
(52.8140, 0.0000, 0.3400)
(74.6781, 0.0000, 0.0953)
The European Put Premium:  2.999


### The Full Recursive Solution for the European Binomial Model

In [5]:
def euroBinomPricerRecursive(S, K, r, v, q, T, n, payoff, verbose = True):
    nodes = n  + 1
    h = T / n
    u = np.exp((r - q) * h + v * np.sqrt(h))
    d = np.exp((r - q) * h - v * np.sqrt(h))
    pu = (np.exp((r - q) * h) - d) / (u - d)
    pd = 1.0 - pu
    disc = np.exp(-r * h)
    
    
    ## Arrays to store the spot prices and option values
    Ct = np.empty(nodes)
    St = np.empty(nodes)
    
    for i in range(nodes):
        St[i] = S * (u ** (n - i)) * (d ** i)
        Ct[i] = payoff(St[i], K)
    
    if verbose:
        print(Ct)
        
    for t in range((n - 1), -1, -1):
        for j in range(t+1):
            Ct[j] = disc * (pu * Ct[j] + pd * Ct[j+1])
            # St[j] = St[j] / u
            # Ct[j] = np.maximum(Ct[j], early payoff)
            print(Ct)
            
    return Ct[0]
            
    

In [6]:
putPrc = euroBinomPricerRecursive(S, K, r, v, q, T, n, putPayoff, verbose = True)
putPrc

[ 0.          0.          2.6487273  13.58434506]
[ 0.          0.          2.6487273  13.58434506]
[ 0.          1.40091085  2.6487273  13.58434506]
[ 0.          1.40091085  8.36287205 13.58434506]
[ 0.74094121  1.40091085  8.36287205 13.58434506]
[ 0.74094121  5.04622553  8.36287205 13.58434506]
[ 2.99850712  5.04622553  8.36287205 13.58434506]


2.9985071167431516

#### Using a Two-Dimensional Array

In [33]:
def euroBinomPricerRecursiveMatrix(S, K, r, v, q, T, n, payoff, verbose = True):
    nodes = n  + 1
    h = T / n
    #u = np.exp((r - q) * h + v * np.sqrt(h))
    u = 1.3
    #d = np.exp((r - q) * h - v * np.sqrt(h))
    d = .8
    pu = (np.exp((r - q) * h) - d) / (u - d)
    pd = 1.0 - pu
    disc = np.exp(-r * h)
    
    ## Arrays to store the spot prices and option values
    Ct = np.zeros((nodes, n+1))
    St = np.zeros((nodes, n+1))
    Dt = np.zeros((nodes, n+1))
    Bt = np.zeros((nodes, n+1))
    for i in range(nodes):
        St[i, n] = S * (u**(n-i)) * (d**i)
        Ct[i, n] = payoff(St[i, n], K)
    

    for t in range((n-1), -1, -1):
        for j in range(t+1):
            St[j, t] = St[j, t+1] / u
            Ct[j, t] = disc * ((pu * Ct[j, t+1]) + (pd * Ct[j+1, t+1]))
            Dt[j, t] = np.exp(-q * h) * (Ct[j, t+1] - Ct[j+1, t+1]) / (St[j, t] * (u - d))
            Bt[j, t] = np.exp(-r * h) * (((u * Ct[j+1, t+1]) - (d * Ct[j, t+1])) / (u - d))

    if verbose:
        print(np.around(St, 3))
        print("\n")
        print(np.around(Ct, 3))
        print("\n")
        print(np.around(Dt, 3))
        print("\n")
        print(np.around(Bt, 3))
        print("\n")            
            
    return Ct[0,0]

In [34]:

S = 41.0
K = 40.0
v = 0.30
r = 0.08
T = 1.0
q = 0.0
n = 3

callPrc = euroBinomPricerRecursiveMatrix(S, K, r, v, q, T, n, callPayoff, verbose = True)

[[41.    53.3   69.29  90.077]
 [ 0.    32.8   42.64  55.432]
 [ 0.     0.    26.24  34.112]
 [ 0.     0.     0.    20.992]]


[[ 9.137 17.041 30.343 50.077]
 [ 0.     3.016  6.823 15.432]
 [ 0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.   ]]


[[0.684 0.883 1.    0.   ]
 [0.    0.416 0.724 0.   ]
 [0.    0.    0.    0.   ]
 [0.    0.    0.    0.   ]]


[[-18.913 -29.999 -38.947   0.   ]
 [  0.    -10.629 -24.041   0.   ]
 [  0.      0.      0.      0.   ]
 [  0.      0.      0.      0.   ]]




### __Problem 1a)__

In [35]:
#Problem 1a)

S = 100.0
K = 105.0
v = 0.30
r = 0.08
T = .5
q = 0.0
n = 1
callPrc = euroBinomPricerRecursiveMatrix(S, K, r, v, q, T, n, callPayoff, verbose = True)
callPrc 

[[100. 130.]
 [  0.  80.]]


[[11.568 25.   ]
 [ 0.     0.   ]]


[[0.5 0. ]
 [0.  0. ]]


[[-38.432   0.   ]
 [  0.      0.   ]]




11.568422433907068

### __Problem 1b)__

In [36]:
#Problem 1b)

S = 100.0
K = 105.0
v = 0.30
r = 0.08
T = .5
q = 0.0
n = 1
putPrc = euroBinomPricerRecursiveMatrix(S, K, r, v, q, T, n, putPayoff, verbose = True)
putPrc

[[100. 130.]
 [  0.  80.]]


[[12.451  0.   ]
 [ 0.    25.   ]]


[[-0.5  0. ]
 [ 0.   0. ]]


[[62.451  0.   ]
 [ 0.     0.   ]]




12.45131354490101

### __Problem 2a)__

In [38]:
#Problem 2a)

S = 100.0
K = 95.0
v = 0.30
r = 0.08
T = .5
q = 0.0
n = 1
putPrc = euroBinomPricerRecursiveMatrix(S, K, r, v, q, T, n, putPayoff, verbose = True)
putPrc 

[[100. 130.]
 [  0.  80.]]


[[ 7.471  0.   ]
 [ 0.    15.   ]]


[[-0.3  0. ]
 [ 0.   0. ]]


[[37.471  0.   ]
 [ 0.     0.   ]]




7.470788126940606

### __Problem2b)__

In [39]:
#Problem 2b)

S = 100.0
K = 95.0
v = 0.30
r = 0.08
T = .5
q = 0.0
n = 1
callPrc = euroBinomPricerRecursiveMatrix(S, K, r, v, q, T, n, callPayoff, verbose = True)
callPrc 

[[100. 130.]
 [  0.  80.]]


[[16.196 35.   ]
 [ 0.     0.   ]]


[[0.7 0. ]
 [0.  0. ]]


[[-53.804   0.   ]
 [  0.      0.   ]]




16.195791407469894

The arbitrage is to observe short on the call price and gain $.81 on the option.

### __Problem 2c)__

The arbitrage is to long the call price and save $.69 on the option.

### __Problem 3__

In [41]:
#Problem 3)

S = 100.0
K = 95.0
v = 0.30
r = 0.08
T = 1
q = 0.0
n = 2
callPrc = euroBinomPricerRecursiveMatrix(S, K, r, v, q, T, n, callPayoff, verbose = True)
callPrc 

[[100. 130. 169.]
 [  0.  80. 104.]
 [  0.   0.  64.]]


[[19.994 38.725 74.   ]
 [ 0.     4.165  9.   ]
 [ 0.     0.     0.   ]]


[[0.691 1.    0.   ]
 [0.    0.225 0.   ]
 [0.    0.    0.   ]]


[[-49.127 -91.275   0.   ]
 [  0.    -13.835   0.   ]
 [  0.      0.      0.   ]]




19.993693459343987

## __Problem 4__

#### __Price $80__

In [42]:
#Problem 4)

S = 80.0
K = 95.0
v = 0.30
r = 0.08
T = 1
q = 0.0
n = 3
callPrc = euroBinomPricerRecursiveMatrix(S, K, r, v, q, T, n, callPayoff, verbose = True)
callPrc 

[[ 80.   104.   135.2  175.76]
 [  0.    64.    83.2  108.16]
 [  0.     0.    51.2   66.56]
 [  0.     0.     0.    40.96]]


[[11.081 21.971 42.7   80.76 ]
 [ 0.     2.572  5.818 13.16 ]
 [ 0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.   ]]


[[0.485 0.709 1.    0.   ]
 [0.    0.182 0.316 0.   ]
 [0.    0.    0.    0.   ]
 [0.    0.    0.    0.   ]]


[[-27.716 -51.793 -92.5     0.   ]
 [  0.     -9.064 -20.502   0.   ]
 [  0.      0.      0.      0.   ]
 [  0.      0.      0.      0.   ]]




11.080555851351969

### __Price $90__

In [43]:
S = 90.0
K = 95.0
v = 0.30
r = 0.08
T = 1
q = 0.0
n = 3
callPrc = euroBinomPricerRecursiveMatrix(S, K, r, v, q, T, n, callPayoff, verbose = True)
callPrc 

[[ 90.   117.   152.1  197.73]
 [  0.    72.    93.6  121.68]
 [  0.     0.    57.6   74.88]
 [  0.     0.     0.    46.08]]


[[ 17.193  32.619  59.6   102.73 ]
 [  0.      5.215  11.795  26.68 ]
 [  0.      0.      0.      0.   ]
 [  0.      0.      0.      0.   ]]


[[0.609 0.817 1.    0.   ]
 [0.    0.328 0.57  0.   ]
 [0.    0.    0.    0.   ]
 [0.    0.    0.    0.   ]]


[[-37.616 -62.99  -92.5     0.   ]
 [  0.    -18.376 -41.565   0.   ]
 [  0.      0.      0.      0.   ]
 [  0.      0.      0.      0.   ]]




17.193212548696497

### __Price $110__

In [44]:
S = 110.0
K = 95.0
v = 0.30
r = 0.08
T = 1
q = 0.0
n = 3
callPrc = euroBinomPricerRecursiveMatrix(S, K, r, v, q, T, n, callPayoff, verbose = True)
callPrc 

[[110.   143.   185.9  241.67]
 [  0.    88.   114.4  148.72]
 [  0.     0.    70.4   91.52]
 [  0.     0.     0.    56.32]]


[[ 29.419  53.917  93.4   146.67 ]
 [  0.     10.5    23.75   53.72 ]
 [  0.      0.      0.      0.   ]
 [  0.      0.      0.      0.   ]]


[[0.789 0.974 1.    0.   ]
 [0.    0.54  0.939 0.   ]
 [0.    0.    0.    0.   ]
 [0.    0.    0.    0.   ]]


[[-57.416 -85.383 -92.5     0.   ]
 [  0.    -37.    -83.69    0.   ]
 [  0.      0.      0.      0.   ]
 [  0.      0.      0.      0.   ]]




29.418525943385543

### __Price $120__

In [45]:
S = 120.0
K = 95.0
v = 0.30
r = 0.08
T = 1
q = 0.0
n = 3
callPrc = euroBinomPricerRecursiveMatrix(S, K, r, v, q, T, n, callPayoff, verbose = True)
callPrc 

[[120.   156.   202.8  263.64]
 [  0.    96.   124.8  162.24]
 [  0.     0.    76.8   99.84]
 [  0.     0.     0.    61.44]]


[[ 37.345  65.934 110.3   168.64 ]
 [  0.     15.417  32.3    67.24 ]
 [  0.      0.      2.14    4.84 ]
 [  0.      0.      0.      0.   ]]


[[0.842 1.    1.    0.   ]
 [0.    0.628 1.    0.   ]
 [0.    0.    0.126 0.   ]
 [0.    0.    0.    0.   ]]


[[-63.688 -90.066 -92.5     0.   ]
 [  0.    -44.903 -92.5     0.   ]
 [  0.      0.     -7.54    0.   ]
 [  0.      0.      0.      0.   ]]




37.34516043327331

### __Price $130__

In [46]:
S = 130.0
K = 95.0
v = 0.30
r = 0.08
T = 1
q = 0.0
n = 3
callPrc = euroBinomPricerRecursiveMatrix(S, K, r, v, q, T, n, callPayoff, verbose = True)
callPrc 

[[130.   169.   219.7  285.61]
 [  0.   104.   135.2  175.76]
 [  0.     0.    83.2  108.16]
 [  0.     0.     0.    66.56]]


[[ 46.576  78.934 127.2   190.61 ]
 [  0.     21.971  42.7    80.76 ]
 [  0.      0.      5.818  13.16 ]
 [  0.      0.      0.      0.   ]]


[[0.876 1.    1.    0.   ]
 [0.    0.709 1.    0.   ]
 [0.    0.    0.316 0.   ]
 [0.    0.    0.    0.   ]]


[[-67.351 -90.066 -92.5     0.   ]
 [  0.    -51.793 -92.5     0.   ]
 [  0.      0.    -20.502   0.   ]
 [  0.      0.      0.      0.   ]]




46.57605994755165

Delta increases at a decreasing rate as the spot price increases. 

## __Problem 5__

#### __A)__

In [49]:
def euroBinomPricerRecursiveMatrix1(S, K, r, v, q, T, n, payoff, verbose = True):
    nodes = n  + 1
    h = T / n
    u = np.exp((r - q) * h + v * np.sqrt(h))
    #u = 1.3
    d = np.exp((r - q) * h - v * np.sqrt(h))
    #d = .8
    pu = (np.exp((r - q) * h) - d) / (u - d)
    pd = 1.0 - pu
    disc = np.exp(-r * h)
    
    ## Arrays to store the spot prices and option values
    Ct = np.zeros((nodes, n+1))
    St = np.zeros((nodes, n+1))
    Dt = np.zeros((nodes, n+1))
    Bt = np.zeros((nodes, n+1))
    for i in range(nodes):
        St[i, n] = S * (u**(n-i)) * (d**i)
        Ct[i, n] = payoff(St[i, n], K)
    

    for t in range((n-1), -1, -1):
        for j in range(t+1):
            St[j, t] = St[j, t+1] / u
            Ct[j, t] = disc * ((pu * Ct[j, t+1]) + (pd * Ct[j+1, t+1]))
            Dt[j, t] = np.exp(-q * h) * (Ct[j, t+1] - Ct[j+1, t+1]) / (St[j, t] * (u - d))
            Bt[j, t] = np.exp(-r * h) * (((u * Ct[j+1, t+1]) - (d * Ct[j, t+1])) / (u - d))

    if verbose:
        print(np.around(St, 3))
        print("\n")
        print(np.around(Ct, 3))
        print("\n")
        print(np.around(Dt, 3))
        print("\n")
        print(np.around(Bt, 3))
        print("\n")            
            
    return Ct[0,0]

In [50]:
S = 100.0
K = 95.0
v = 0.30
r = 0.08
T = 1
q = 0.0
n = 3
callPrc = euroBinomPricerRecursiveMatrix(S, K, r, v, q, T, n, callPayoff, verbose = True)
callPrc 

[[100.    122.125 149.144 182.142]
 [  0.     86.369 105.478 128.815]
 [  0.      0.     74.596  91.101]
 [  0.      0.      0.     64.428]]


[[18.283 33.149 56.644 87.142]
 [ 0.     6.69  15.04  33.815]
 [ 0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.   ]]


[[0.74  0.953 1.    0.   ]
 [0.    0.487 0.897 0.   ]
 [0.    0.    0.    0.   ]
 [0.    0.    0.    0.   ]]


[[-55.719 -83.207 -92.5     0.   ]
 [  0.    -35.375 -79.532   0.   ]
 [  0.      0.      0.      0.   ]
 [  0.      0.      0.      0.   ]]




18.28255220737056

No excercise on the option

### __B)__

In [51]:
S = 100.0
K = 95.0
v = 0.30
r = 0.08
T = 1
q = 0.0
n = 3
callPrc = euroBinomPricerRecursiveMatrix(S, K, r, v, q, T, n, callPayoff, verbose = True)
callPrc 

[[100.    122.125 149.144 182.142]
 [  0.     86.369 105.478 128.815]
 [  0.      0.     74.596  91.101]
 [  0.      0.      0.     64.428]]


[[18.283 33.149 56.644 87.142]
 [ 0.     6.69  15.04  33.815]
 [ 0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.   ]]


[[0.74  0.953 1.    0.   ]
 [0.    0.487 0.897 0.   ]
 [0.    0.    0.    0.   ]
 [0.    0.    0.    0.   ]]


[[-55.719 -83.207 -92.5     0.   ]
 [  0.    -35.375 -79.532   0.   ]
 [  0.      0.      0.      0.   ]
 [  0.      0.      0.      0.   ]]




18.28255220737056

### __C)__

In [52]:
S = 100.0
K = 95.0
v = 0.30
r = 0.08
T = 1
q = 0.0
n = 3
putPrc = euroBinomPricerRecursiveMatrix(S, K, r, v, q, T, n, putPayoff, verbose = True)
putPrc 

[[100.    122.125 149.144 182.142]
 [  0.     86.369 105.478 128.815]
 [  0.      0.     74.596  91.101]
 [  0.      0.      0.     64.428]]


[[ 5.979  1.091  0.     0.   ]
 [ 0.    10.387  2.062  0.   ]
 [ 0.     0.    17.904  3.899]
 [ 0.     0.     0.    30.572]]


[[-0.26  -0.047  0.     0.   ]
 [ 0.    -0.513 -0.103  0.   ]
 [ 0.     0.    -1.     0.   ]
 [ 0.     0.     0.     0.   ]]


[[31.977  6.859  0.     0.   ]
 [ 0.    54.691 12.968  0.   ]
 [ 0.     0.    92.5    0.   ]
 [ 0.     0.     0.     0.   ]]




5.97860511410097

In [58]:
parity = S - (K * np.exp(-r*T))
parity

12.3039470932696

In [64]:
parity1 = callPrc - putPrc
parity1

12.30394709326959

Put-call parity will hold.

### __D)__

In [72]:

def euroBinomPricerRecursive(S, K, r, v, q, T, n, payoff, verbose = False):
    nodes = n  + 1
    h = T / n
    u = np.exp((r - q) * h + v * np.sqrt(h))
    d = np.exp((r - q) * h - v * np.sqrt(h))
    pu = (np.exp((r - q) * h) - d) / (u - d)
    pd = 1.0 - pu
    disc = np.exp(-r * h)
   
   
   ## Arrays to store the spot prices and option values
    Ct = np.empty(nodes)
    St = np.empty(nodes)
   
    for i in range(nodes):
        St[i] = S * (u ** (n - i)) * (d ** i)
        Ct[i] = payoff(St[i], K)
   
    if verbose:
        print(Ct)
       
    for t in range((n - 1), -1, -1):
        for j in range(t+1):
            Ct[j] = disc * (pu * Ct[j] + pd * Ct[j+1])
            St[j] = St[j] / u
            Ct[j] = np.maximum(Ct[j], K - St[j])
            print(Ct)
           
    return Ct[0]
def euroBinomPricerRecursiveMatrix(S, K, r, v, q, T, n, payoff, verbose = False):
    nodes = n  + 1
    h = T / n
    u = np.exp((r - q) * h + v * np.sqrt(h))
    d = np.exp((r - q) * h - v * np.sqrt(h))
    pu = (np.exp((r - q) * h) - d) / (u - d)
    pd = 1.0 - pu
    disc = np.exp(-r * h)
   
   ## Arrays to store the spot prices and option values
    Ct = np.zeros((nodes, n+1))
    St = np.zeros((nodes, n+1))
    Dt = np.zeros((nodes, n+1))
   
    for i in range(nodes):
        St[i, n] = S * (u**(n-i)) * (d**i)
        Ct[i, n] = payoff(St[i, n], K)
   

    for t in range((n-1), -1, -1):
        for j in range(t+1):
            St[j, t] = St[j, t+1] / u
            Ct[j, t] = disc * ((pu * Ct[j, t+1]) + (pd * Ct[j+1, t+1]))
            Dt[j, t] = np.exp(-q * h) * (Ct[j, t+1] - Ct[j+1, t+1]) / (St[j, t] * (u - d))
         
    return Ct[0,0]

In [73]:
S = 100
K = 95
r = 0.08
v = 0.30
q = 0.0
T = 1
n = 3

In [74]:
callPrcA = euroBinomPricerRecursive(S, K, r, v, q, T, n, callPayoff,  verbose = False)
callPrcE = euroBinomPricerRecursiveMatrix(S, K, r, v, q, T, n, callPayoff,  verbose = False)
putPrcA = euroBinomPricerRecursive(S, K, r, v, q, T, n, putPayoff,  verbose = False)
putPrcE = euroBinomPricerRecursiveMatrix(S, K, r, v, q, T, n, putPayoff,  verbose = False)

print(f"The European Call Premium: {callPrcE : 0.3f} The American call is {callPrcE : 0.3f}")
print(f"The European Put Premium: {putPrcE : 0.3f} The American put is {putPrcA : 0.3f}")

[56.64406241 33.8147424   0.          0.        ]
[56.64406241 15.04032855  0.          0.        ]
[56.64406241 15.04032855 20.40351726  0.        ]
[33.14931753 15.04032855 20.40351726  0.        ]
[33.14931753 17.48114202 20.40351726  0.        ]
[23.99012609 17.48114202 20.40351726  0.        ]
[ 0.          0.          3.89933488 30.57157332]
[ 0.          2.06235672  3.89933488 30.57157332]
[ 0.          2.06235672 20.40351726 30.57157332]
[ 1.09077967  2.06235672 20.40351726 30.57157332]
[ 1.09077967 11.7087201  20.40351726 30.57157332]
[ 6.67790123 11.7087201  20.40351726 30.57157332]
The European Call Premium:  18.283 The American call is  18.283
The European Put Premium:  5.979 The American put is  6.678


## __Problem 6__

### __A)__

In [77]:
S = 40
K = 40
r = 0.08
v = 0.30
q = 0.0
T = .5
n = 3
h = T / n

In [78]:
u1 = np.exp((r - q) * h + v * np.sqrt(h))
d1 = np.exp((r - q) * h - v * np.sqrt(h))
print(u1,d1)

1.1454617381452392 0.8966038495199921


### __B)__

In [80]:
callPrcA = euroBinomPricerRecursive(S, K, r, v, q, T, n, callPayoff,  verbose = False)
callPrcE = euroBinomPricerRecursiveMatrix(S, K, r, v, q, T, n, callPayoff,  verbose = False)
putPrcA = euroBinomPricerRecursive(S, K, r, v, q, T, n, putPayoff,  verbose = False)
putPrcE = euroBinomPricerRecursiveMatrix(S, K, r, v, q, T, n, putPayoff,  verbose = False)

print(f"The European Call Premium: {callPrcE : 0.3f} The American call is {callPrcE : 0.3f}")
print(f"The European Put Premium: {putPrcE : 0.3f} The American put is {putPrcA : 0.3f}")

[13.01309727  7.05673217  0.          0.        ]
[13.01309727  3.26869392  0.          0.        ]
[13.01309727  3.26869392  7.84406148  0.        ]
[7.73902974 3.26869392 7.84406148 0.        ]
[7.73902974 5.6208478  7.84406148 0.        ]
[6.52754713 5.6208478  7.84406148 0.        ]
[ 0.          0.          3.16660277 11.16886174]
[ 0.          1.65788424  3.16660277 11.16886174]
[ 0.          1.65788424  7.84406148 11.16886174]
[ 0.86799019  1.65788424  7.84406148 11.16886174]
[ 0.86799019  4.8747172   7.84406148 11.16886174]
[ 2.95422756  4.8747172   7.84406148 11.16886174]
The European Call Premium:  4.377 The American call is  4.377
The European Put Premium:  2.809 The American put is  2.954
